# Assignment 2: Language Modelling in Hangman

Student Name: Arya Araban

Student ID: 1439683

## General info

<b>Due date</b>: Thursday, 6 April 2023 1pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 8% of mark for class (with 7% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See "Using Jupyter Notebook and Python page" on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, Gensim, Keras and Pytorch. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3.8</b>; libraries/packages should be in their latest version that is compatible with python3.8.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=https://www.nltk.org/book/>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place. In regards to the use of artificial intelligence tools in the context of academic integrity, please see the university's statement <a href="https://academicintegrity.unimelb.edu.au/plagiarism-and-collusion/artificial-intelligence-tools-and-technologies">here</a>.

# Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character-level n-gram language models. Your objective is to create an automatic player which makes the fewest mistakes.

# The Hangman Game (7 marks)

**Instructions**: The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. **No implementation is needed.**

In [1]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [2]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to `True`. When submitting your solution, set to `False` so we can automatically run the whole notebook using `Run All`.

In [3]:
interactive = False

<b>For your testing:</b>

You can play the game interactively using the following command:

In [4]:
if interactive:
    hangman('whatever', human, 8, True)

### Question 1 (1.0 mark)

**Instructions**: We will use the words in NLTK's Brown corpus for training an artificial intelligence guessing algorithm, and for evaluating the quality of the algorithm.

Your first task is to compute the number of **unique word types** occurring in the Brown corpus, using `nltk.corpus.brown` and the `words` method, and select only words that are **entirely comprised of alphabetic characters**. You should also **lowercase the words**. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the  training set. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

**Task**: Collect all unique word types from the Brown corpus, and produce `training_set` and `test_set`, 2 lists that contain 2 disjointed sets of words. Both `training_set` and `test_set` should be a python `list` (as initialised in the code). `test_set` must contain exactly 1000 word types.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [5]:
import nltk
from nltk.corpus import brown
import numpy as np

nltk.download('brown')
np.random.seed(1)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []


###
# Your answer BEGINS HERE
###



word_types = set() # using set to ensure only unique words will be in it 
for word in brown.words():
    if all(ord('a') <= ord(char.lower()) <= ord('z') for char in word):
        word_types.add(word.lower())
        
word_types = np.array(list(word_types))
np.random.shuffle(word_types)

test_set = list(word_types[:1000])
training_set = list(word_types[1000:])



###
# Your answer ENDS HERE
###

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Number of word types in test = 1000
Number of word types in train = 39234


<b>For your testing:</b>

In [6]:
assert(len(training_set) > 35000 and len(training_set) < 45000)
assert(len(test_set) == 1000)

**Play the game**:

Let's see how good you are at this game! Try to guess a random word from the test set. It is surprisingly difficult (and addictive)! Don't forget to set `interactive = True`.

In [7]:
#play hangman using random words from test set
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

### Question 2 (1.0 mark)

**Instructions**: To set a baseline, your first AI attempt will be a trivial **random method**. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `a ... z` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses).

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test` is provided to you. 

**Task**: Complete the `random_guesser` method. It should return a random character from the English alphabets.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [8]:
def test_guesser(guesser, test):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [9]:
def random_guesser(mask, guessed, **kwargs):
    
    ###
    # Your answer BEGINS HERE
    ###
    
    available_chars = set('abcdefghijklmnopqrstuvwxyz') - guessed
    return np.random.choice(list(available_chars))


    ###
    # Your answer ENDS HERE
    ###

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser, test_set)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = grunted
Number of mistakes made by the random guesser = 19

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.767


<b>For your testing:</b>

In [10]:
assert(result > 10 and result < 20)

### Question 3 (1.0 mark)

**Instructions:** As your first real AI, you should train a **unigram language model** over the training set. This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

**Task**: Collect the frequencies of characters and store them in `unigram_counts` (use the first answer space). Complete the `unigram_guesser` method. Note that it takes `unigram_counts` as an additional argument (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [11]:
unigram_counts = {}

###
# Your answer BEGINS HERE
###
from copy import deepcopy


for word in training_set:

    for char in word:

         unigram_counts[char] = unigram_counts.get(char, 0) + 1 # if key "char" doesn't already exist, assign default value 0 to it 
      
# Normalizing for probability
unigram_probs = deepcopy(unigram_counts)

total = sum(unigram_probs.values())
for char in unigram_probs:
    unigram_probs[char] /= total # unigram MLE probability 

###
# Your answer ENDS HERE
###

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):

    ###
    # Your answer BEGINS HERE
    ###

    available_chars = set('abcdefghijklmnopqrstuvwxyz') - guessed
    return max(available_chars, key=lambda char: unigram_probs[char])
    
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_guesser, test_set)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.188


<b>For your testing:</b>

In [12]:
assert(result > 5 and result < 15)

### Question 4 (1.0 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different lengths tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e.,  having a **different unigram model for each length**. You will need to be a little careful at test time, to be robust to the situation that you encounter a word length that you didn't see in training. In such a case, your method should behave like the previous `unigram_guesser` in Question 3 (i.e., it guesses characters based on unigram frequencies, unconditioned by the word length).

**Task**: Collect the frequencies of characters conditioned on the word length and store them in `unigram_counts_by_length` (use the first answer space). Complete the `unigram_length_guesser` method (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [13]:
unigram_counts_by_length = {}

###
# Your answer BEGINS HERE
###




for word in training_set:

    length = len(word)

    if length not in unigram_counts_by_length:
        unigram_counts_by_length[length] = {}

    for char in word:

        unigram_counts_by_length[length][char] = unigram_counts_by_length[length].get(char, 0) + 1

# Normalizing for probability
unigram_probs_by_length = deepcopy(unigram_counts_by_length)

for length in unigram_probs_by_length:

    total = sum(unigram_probs_by_length[length].values())

    for char in unigram_probs_by_length[length]:
        unigram_probs_by_length[length][char] /= total # unigram MLE probability 




###
# Your answer ENDS HERE
###

def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):
    ###
    # Your answer BEGINS HERE
    ###

    length = len(mask)
    available_chars = set('abcdefghijklmnopqrstuvwxyz') - guessed
   
    if length not in unigram_counts_by_length:  # If no data on this length of the secret word, use the unigram_guesser
       return max(available_chars, key=lambda char: unigram_probs[char]) #falling back to unigram model
       

    
      
    return max(available_chars, key=lambda char: unigram_probs_by_length[length].get(char, 0))

    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_length_guesser, test_set)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.18


<b>For your testing:</b>

In [14]:
assert(result > 5 and result < 15)

### Question 5 (1.0 mark)

**Instructions:** Now for the next challenge, you'll build a **bigram language model** over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

The task here is to develop a bigram language model over characters, and train it over the training words. Remember to be careful when handling the start of each word properly, e.g., by padding with a special starting symbol such as `$`. Do we also need a special ending symbol? That's for you to decide.

Your bigram guesser should apply your language model to each blank position in the secret word by using its left context character. For example, in the partial word `e _ c _ b _ _` we know the left context for the first three blanks, but have no known left context for the last blank. In the case for the last blank, you should revert to using a unigram language model (since there's no context for us to use the bigram model). You should sum up the probability distribution (over all alphabets from <i>a</i> to <i>z</i>) for the 4 blanks, and select the alphabet with the highest probability that hasn't been guessed.

**Note**:
- When backing-off to the unigram language model, you **must use the vanilla unigram language model that you have built in Q3**. Do not use the length-based unigram language model, or any fancy variant, or you will lose marks.
- You should build a **standard bigram language model**; i.e. do not do anything complicated like a bidirectional bigram language model.

**Task**: Collect frequency counts that are necessary for building a bigram language model and store them in bigram_counts; feel free to add new objects if needed (use the first answer space). Complete the `bigram_guesser` method (use the second answer space). Note that the method currently only has one additional argument (`bigram_counts`), but you are free to add additional arguments.

In [15]:
bigram_counts = {}

###
# Your answer BEGINS HERE
###


def get_bigram_prob(bigram, bigram_counts, unigram_counts):
    # function which returns MLE probability for certain bigram.

    first_char = bigram[0]

    bg_count = bigram_counts.get(bigram,0) 

    ug_count = unigram_counts[first_char]

    if not ug_count:
      return 0 

    return bg_count / ug_count 
    
    

start_symbol = '$'
unigram_counts[start_symbol] = len(training_set) # each word in the training set has the start symbol 

for word in training_set:

    word = start_symbol + word 
    
    for i in range(len(word)-1):
        bigram = word[i:i+2]

        bigram_counts[bigram] =  bigram_counts.get(bigram, 0) + 1




###
# Your answer ENDS HERE
###
    

def bigram_guesser(mask, guessed, bigram_counts=bigram_counts): # add extra arguments if needed
    
    prob_dist = {}
  
    available_chars = set('abcdefghijklmnopqrstuvwxyz') - guessed
    
    for i in range(len(mask)):
        
        if mask[i] != '_': # If non-blank, continue
            continue
        
        
        prob_dist[i] = {} # prob distribution for current blank
        
        
        if i == 0:
            left_char = start_symbol
        else:
            left_char = mask[i-1]
        

        for char in available_chars:

            if left_char == "_":
              prob_dist[i][char] = unigram_probs[char]
               
            else:
              bigram = left_char + char
              prob_dist[i][char] = get_bigram_prob(bigram, bigram_counts, unigram_counts)
          
           
                
        
            
    
    # Sum up the probability distribution over all blanks
    final_prob_dist = {}

    for i in prob_dist.keys():
        for char in prob_dist[i]:
            final_prob_dist[char] = final_prob_dist.get(char, 0) + prob_dist[i][char]
    
    
    guess = max(available_chars, key=lambda char: final_prob_dist.get(char, 0))
    return guess



result = test_guesser(bigram_guesser, test_set)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  8.613


### Question 6 (1.5 mark)

**Instructions:** You should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, ngram models and bidirectional models (lecture 8). Have fun!

You will be marked based on the performance of your AI model, using a pre-made training and test set (created using a secret seed). Let x be the average number of mistakes in the test set, you will score:
* 1.5 mark if x < 7.6
* 1.0 mark if 7.6 <= x < 8.0
* 0.5 mark if 8.0 <= x < 8.5
* 0.0 mark if x >= 8.5

Note:
* When testing your AI model's performance, you may want to consider trying different training/test splits (using different seeds) to have a better understanding of its performance, as there will be some variance to its performance depending on the training/test split.
* Your code must run under 3 minutes on Codalab; program that runs longer than that will be terminated and you will score 0.

**Task** Complete the `my_amazing_ai_guesser` method, which implements a better language model for hangman.

In [20]:
###
# Your answer BEGINS HERE
###

def get_bigram_prob_reversed(bigram, bigram_counts, unigram_counts):
    # To compute the bigram probabilities for a right-to-left direction,
    # we divide by the unigram counts of the second character instead of the first
    
    second_char = bigram[1]

    bg_count = bigram_counts.get(bigram,0) 

    ug_count = unigram_counts[second_char]

    if not ug_count:
      return 0 

    return bg_count / ug_count

def geo_mean(iterable):
    # calculate geo-mean, used for combining the forward and backward bigram probabilities. 
    a = np.array(iterable)
    return a.prod()**(1.0/len(a))
    

start_symbol = '$'
end_symbol = '#'

unigram_counts[start_symbol] = len(training_set) # each word in the training set has the start symbol 
unigram_counts[end_symbol] = len(training_set) # each word in the training set has the end symbol 

for word in training_set:

    word = start_symbol + word + end_symbol
    
    for i in range(len(word)-1):
        bigram = word[i:i+2]

        bigram_counts[bigram] =  bigram_counts.get(bigram, 0) + 1



def my_amazing_ai_guesser(mask, guessed):
    
    prob_dist = {}
  
    available_chars = set('abcdefghijklmnopqrstuvwxyz') - guessed
    
    for i in range(len(mask)):
        
        if mask[i] != '_': # If non-blank, continue
            continue
        
        
        prob_dist[i] = {} # prob distribution for current blank
        
        
        if i == 0:
            left_char = start_symbol
        else:
            left_char = mask[i-1]

        if i==len(mask)-1: 
            right_char = end_symbol

        else: 
            right_char = mask[i+1]

        for char in available_chars:

            if left_char == "_":

              try: 
                left_prob = unigram_probs_by_length[len(mask)][char]
              except(KeyError): # in rare case of key_error, go back to unigram model
                left_prob = unigram_probs[char]

            else:
              bigram_left = left_char + char
              left_prob = get_bigram_prob(bigram_left, bigram_counts, unigram_counts)


            if right_char == "_": 
              try: 
                right_prob = unigram_probs_by_length[len(mask)][char]
              except(KeyError):
                right_prob = unigram_probs[char]

            else:
              bigram_right =  char + right_char
              right_prob = get_bigram_prob_reversed(bigram_right, bigram_counts, unigram_counts)


            prob_dist[i][char] = geo_mean([left_prob, right_prob])
                
        
            
    
    # Sum up the probability distribution over all blanks
    final_prob_dist = {}

    for i in prob_dist.keys():
        for char in prob_dist[i]:
            final_prob_dist[char] = final_prob_dist.get(char, 0) + prob_dist[i][char]
    
    
    guess = max(available_chars, key=lambda char: final_prob_dist.get(char, 0))
    return guess


###
# Your answer ENDS HERE
###

result = test_guesser(my_amazing_ai_guesser, test_set) 
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  7.835


### Question 7 (0.5 mark)

**Instructions:** Explain your approach and discuss your result below. Please keep your explanation to a short paragraph.

##### Your answer BEGINS HERE

I implemented bidirectional bigram in such a way that for each blank, we consider both the character in front of it, and the character behind it, and calculate the bigram probabilites seperately. Then, we have to come up with a way to mix these two probabilites. I tried various methods such as weighted mean, geographic mean, and harmonic mean, and got the best results from geographic mean (although harmonic mean worked well too, but it is slower to compute) with my results generally getting the Average number of incorrect guesses as less than 8. Note that I also made some minor tweaks compared to the previous bigram implementation such as falling back to unigram_length_guesser model instead of unigram_guesser when two consecutive blank spaces are seen.


##### Your answer ENDS HERE